In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, PrecisionRecallDisplay, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from itertools import cycle
import warnings
import lightgbm as lgb
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, StackingClassifier
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import label_binarize
from io import BytesIO
from zipfile import ZipFile
import zipfile

In [2]:
zip_file_path = 'Dutch_data_processed_onehot.zip'
csv_file_inside_zip = 'Dutch_data_processed_onehot'

In [3]:
with ZipFile(zip_file_path, 'r') as zip_file:
    with zip_file.open(csv_file_inside_zip) as csv_file:
        df = pd.read_csv(csv_file)

In [4]:
warnings.filterwarnings("ignore")

In [5]:
df

,People_in_house,Gender,Age,Driver_license,Cars_in_house,Mopeds_in_house,Electric_bike,Year,Month,Weekday,...,Motive_Social and recreational,Motive_Tours/hiking,Motive_Visits,Household_composition_Couple,Household_composition_Couple and kids,Household_composition_One person household,Household_composition_Other,Education_Higher education,Education_Other,Education_Secondary
0,1,1,42,1,1,0,0,2018,1,6,...,0,0,0,0,0,1,0,0,0,1
1,1,1,42,1,1,0,0,2018,1,6,...,0,0,0,0,0,1,0,0,0,1
2,1,1,42,1,1,0,0,2018,1,6,...,0,0,1,0,0,1,0,0,0,1
3,2,1,42,1,1,0,0,2018,1,5,...,0,0,0,1,0,0,0,0,0,1
4,2,1,42,1,1,0,0,2018,1,5,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558806,3,0,49,1,1,0,0,2022,12,4,...,0,0,0,0,0,0,1,1,0,0
558807,3,0,49,1,1,0,0,2022,12,4,...,0,0,0,0,0,0,1,1,0,0
558808,3,0,49,1,1,0,0,2022,12,4,...,0,0,0,0,0,0,1,1,0,0
558809,3,0,49,1,1,0,0,2022,12,4,...,0,0,0,0,0,0,1,1,0,0


# 4 labels: Car, Bike, Walk, Public Trasport

In [14]:
drops = ["Mode", "Arrival_area", "Year", "Motive_Tours/hiking", "Household_composition_Couple and kids",
        'Education_Secondary', 'Background_Non Western migration', 'Departure_area_Other', 'Motive_Shopping/Grocery']
X = df.drop(drops, axis=1)
y = df["Mode"]

In [15]:
X

,People_in_house,Gender,Age,Driver_license,Cars_in_house,Mopeds_in_house,Electric_bike,Month,Weekday,Holiday,...,Motive_Go to/return from work,Motive_Other motives,Motive_Service/Care/Business,Motive_Social and recreational,Motive_Visits,Household_composition_Couple,Household_composition_One person household,Household_composition_Other,Education_Higher education,Education_Other
0,1,1,42,1,1,0,0,1,6,0,...,0,0,0,0,0,0,1,0,0,0
1,1,1,42,1,1,0,0,1,6,0,...,0,0,0,0,0,0,1,0,0,0
2,1,1,42,1,1,0,0,1,6,0,...,0,0,0,0,1,0,1,0,0,0
3,2,1,42,1,1,0,0,1,5,0,...,0,0,0,0,0,1,0,0,0,0
4,2,1,42,1,1,0,0,1,5,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558806,3,0,49,1,1,0,0,12,4,0,...,0,0,1,0,0,0,0,1,1,0
558807,3,0,49,1,1,0,0,12,4,0,...,0,0,1,0,0,0,0,1,1,0
558808,3,0,49,1,1,0,0,12,4,0,...,0,0,0,0,0,0,0,1,1,0
558809,3,0,49,1,1,0,0,12,4,0,...,0,0,0,0,0,0,0,1,1,0


In [16]:
X, y = shuffle(X, y, random_state=42)

In [17]:
X

,People_in_house,Gender,Age,Driver_license,Cars_in_house,Mopeds_in_house,Electric_bike,Month,Weekday,Holiday,...,Motive_Go to/return from work,Motive_Other motives,Motive_Service/Care/Business,Motive_Social and recreational,Motive_Visits,Household_composition_Couple,Household_composition_One person household,Household_composition_Other,Education_Higher education,Education_Other
52038,5,1,50,1,2,0,0,7,3,0,...,0,0,0,0,0,0,0,0,0,0
409475,2,1,33,1,1,0,0,9,4,0,...,0,0,0,0,1,1,0,0,1,0
479503,2,1,80,1,1,0,0,11,7,0,...,0,0,0,0,1,1,0,0,0,0
327267,4,1,47,1,2,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
553063,1,1,36,1,1,1,0,6,4,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,4,1,56,1,1,1,0,1,5,0,...,0,0,1,0,0,0,0,0,0,0
259178,4,0,49,1,1,0,1,6,4,0,...,1,0,0,0,0,0,0,0,0,1
365838,3,0,51,1,1,0,0,5,5,0,...,1,0,0,0,0,0,0,0,0,0
131932,2,0,54,1,2,0,0,3,7,0,...,0,0,0,0,1,1,0,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state =42, stratify=y)

In [19]:
y_train_series = pd.Series(y_train)
y_test_series = pd.Series(y_test)

print("Class counts in y_train:")
print(y_train_series.value_counts())

print("\nClass counts in y_test:")
print(y_test_series.value_counts())

Class counts in y_train:
Mode
0    235601
2    127197
1    119630
3     20501
Name: count, dtype: int64

Class counts in y_test:
Mode
0    26179
2    14133
1    13292
3     2278
Name: count, dtype: int64


In [20]:
X_train.shape, X_test.shape

((502929, 30), (55882, 30))

In [13]:
# Combine X_train and y_train back into a single DataFrame
train_data = pd.concat([X_train, y_train], axis=1)

# Calculate the mean number of trips per travel mode using only the training set
mean_trips_per_mode = train_data['Mode'].value_counts().mean()

# Create an empty DataFrame to store the resampled training data
resampled_train_df = pd.DataFrame(columns=X_train.columns)

# For each class in the training set
for mode in train_data['Mode'].unique():
    mode_data = train_data[train_data['Mode'] == mode]
    num_instances = len(mode_data)

    # If the class has less than mean_trips_per_mode cases, sample with replacement and add to the resampled training DataFrame
    if num_instances < mean_trips_per_mode:
        resampled_mode_data = mode_data.sample(n=int(mean_trips_per_mode - num_instances), replace=True)
        resampled_train_df = pd.concat([resampled_train_df, mode_data, resampled_mode_data])

    # If the class has more than mean_trips_per_mode cases, randomly remove instances until there are mean_trips_per_mode cases
    elif num_instances > mean_trips_per_mode:
        resampled_mode_data = mode_data.sample(n=int(num_instances - mean_trips_per_mode))
        resampled_train_df = pd.concat([resampled_train_df, mode_data.drop(resampled_mode_data.index)])

# Shuffle the resampled training DataFrame
resampled_train_df = resampled_train_df.sample(frac=1).reset_index(drop=True)

# Check if each class in the resampled training set has exactly mean_trips_per_mode instances
print(resampled_train_df['Mode'].value_counts())

Mode
0.0    155713
1.0    155713
2.0    155713
3.0    155712
Name: count, dtype: int64


In [15]:
df1 = resampled_train_df

In [16]:
df1.isnull().any()

People_in_house                               False
Gender                                        False
Age                                           False
Driver_license                                False
Cars_in_house                                 False
Mopeds_in_house                               False
Electric_bike                                 False
Month                                         False
Weekday                                       False
Holiday                                       False
Round_trip                                    False
Duration                                      False
Depart_time                                   False
Distance_in_kms                               False
Departure_area_Genderland                     False
Departure_area_North Brabant                  False
Departure_area_North Holland                  False
Departure_area_South Holland                  False
Departure_area_Utrecht                        False
Background_D

In [17]:
df1.shape

(622851, 31)

In [19]:
drops = ["Mode"]
X1 = df1.drop(drops, axis=1)
y1 = df1["Mode"]

In [20]:
X1.shape, y1.shape

((622851, 30), (622851,))

In [21]:
y_train_series = pd.Series(y1)
y_test_series = pd.Series(y_test)

print("Class counts in y_train:")
print(y_train_series.value_counts())

print("\nClass counts in y_test:")
print(y_test_series.value_counts())

Class counts in y_train:
Mode
0.0    155713
1.0    155713
2.0    155713
3.0    155712
Name: count, dtype: int64

Class counts in y_test:
Mode
0    31710
1    17682
2    17353
3     2461
Name: count, dtype: int64


In [21]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_features=3)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
tree_report_test = classification_report(y_test, y_pred)
print(tree_report_test, sep='\n')

              precision    recall  f1-score   support

           0       0.86      0.91      0.88     26179
           1       0.73      0.66      0.69     13292
           2       0.84      0.87      0.86     14133
           3       0.80      0.43      0.56      2278

    accuracy                           0.82     55882
   macro avg       0.81      0.72      0.75     55882
weighted avg       0.82      0.82      0.82     55882

